In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

In [0]:
airlines_path = '/content/drive/My Drive/Thinkful DS (trial) /Data/airlines.csv'
census_divisions_path = '/content/drive/My Drive/Thinkful DS (trial) /Data/census-divisions.csv'
flights_path = '/content/drive/My Drive/Thinkful DS (trial) /Data/flights.csv'
ksprojects_path = '/content/drive/My Drive/Thinkful DS (trial) /Data/ksprojects.csv'
state_pop_path = '/content/drive/My Drive/Thinkful DS (trial) /Data/state-populations.csv'


Kickstarter projects 

 Create a DataFrame from the table named ksprojects. Write code that allows you to answer the following questions. Create a separate cell for each question.

1. How many rows of data are in the DataFrame?
2. What are the names and data types of the columns?
3. Do any of the columns contain null values?
4. Find all successful documentary projects and sort them by the amount pledged. Print the top 10 highest pledges.
5. Create a new column named average_per_backer and set its value to the total amount pledged / number of backers. What happened to the rows with 0 backers?
6. How can this be dealt with?
7. Drop all rows with 0 backers then repeat the previous exercise.
8. Create a crosstab to get a count of records for each combination of state and category.


In [0]:
ksp_df = pd.read_csv(ksprojects_path) 

In [0]:
len(ksp_df)

65418

In [0]:
ksp_df.dtypes

In [0]:
ksp_df.isnull()

In [0]:
# documentaries that are successful
successful_documentary = (ksp_df['category'] == 'Documentary') & (ksp_df['state'] == 'successful')

# the largest ones to print
top_n = 100 

# printing n largest values
ksp_df.iloc[successful_documentary.values].nlargest(top_n, columns='usd_pledged')

In [0]:
# new column average_per_backer 
ksp_with_avgbacker = ksp_df.assign(average_per_backer = ksp_df.usd_pledged/ksp_df.backers)

# Where there were no backers, division by zero error results in NaN. filling these up with "not applicable"
ksp_with_avgbacker['average_per_backer'].fillna('not applicable', inplace = True)
ksp_with_avgbacker.tail()

In [0]:
# drop rows with 0 backers
ksp_without_0backers = ksp_df.iloc[(ksp_df.backers > 0).values]

# Check: no zero backers
(ksp_without_0backers.backers == 0).value_counts()

# adding new column average_per_backer to df with no zero-backers
ksp_without_0backers.assign(average_per_backer = ksp_without_0backers.usd_pledged/ksp_without_0backers.backers)

In [0]:
# crosstab of state and category
pd.crosstab(ksp_df['state'],ksp_df['category'])

category,3D Printing,Academic,Accessories,Action,Animals,Animation,Anthologies,Apparel,Apps,Architecture,Art,Art Books,Audio,Bacon,Blues,Calendars,Camera Equipment,Candles,Ceramics,Children's Books,Childrenswear,Chiptune,Civic Design,Classical Music,Comedy,Comic Books,Comics,Community Gardens,Conceptual Art,Cookbooks,Country & Folk,Couture,Crafts,Crochet,DIY,DIY Electronics,Dance,Design,Digital Art,Documentary,...,R&B,Radio & Podcasts,Ready-to-wear,Residencies,Restaurants,Robots,Rock,Romance,Science Fiction,Sculpture,Shorts,Small Batch,Software,Sound,Space Exploration,Spaces,Stationery,Tabletop Games,Taxidermy,Technology,Television,Textiles,Theater,Thrillers,Translations,Typography,Vegan,Video,Video Art,Video Games,Wearables,Weaving,Web,Webcomics,Webseries,Woodworking,Workshops,World Music,Young Adult,Zines
state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
canceled,17,21,48,14,2,41,4,127,166,10,116,29,8,5,3,10,12,15,7,92,17,1,1,26,37,32,71,9,17,15,40,9,84,2,23,13,20,94,28,261,...,7,10,17,0,52,12,79,2,14,26,203,17,72,9,4,8,3,334,1,169,14,3,65,19,0,2,12,9,3,380,25,3,114,4,119,10,0,31,16,6
failed,59,86,238,79,31,245,34,684,784,86,702,187,46,30,19,26,21,50,24,664,55,0,34,151,181,122,397,43,86,49,236,37,461,18,144,47,114,388,112,1599,...,58,73,89,2,320,36,527,13,52,180,902,186,367,46,32,80,25,606,2,694,110,24,459,70,17,4,40,49,24,1168,71,8,634,28,656,125,12,174,78,26
live,3,3,16,1,0,8,6,37,28,4,17,6,2,1,2,2,1,1,2,20,4,0,1,4,9,13,3,0,2,1,7,1,16,0,5,3,2,13,1,20,...,0,3,0,0,10,1,11,2,3,1,8,6,10,4,0,2,1,41,0,25,3,0,4,2,0,0,2,1,0,26,7,0,23,3,5,4,1,2,1,1
successful,43,43,121,16,11,134,83,211,55,26,559,221,18,8,12,24,33,12,19,385,11,4,18,309,129,231,465,9,69,26,513,5,199,8,38,77,327,224,59,1087,...,19,72,27,1,63,35,683,13,50,122,1292,99,71,40,18,39,11,1223,0,227,31,12,846,31,8,8,31,9,5,427,57,3,73,55,313,38,8,168,23,17
suspended,2,4,2,0,1,1,0,5,5,2,1,1,0,2,0,1,1,0,0,0,0,0,1,2,3,1,3,0,0,1,0,1,4,0,3,0,0,10,1,2,...,0,0,2,0,4,2,2,1,1,2,2,2,2,3,3,0,1,4,0,17,0,0,1,2,0,0,1,1,0,10,2,1,6,0,6,2,1,1,0,0
undefined,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


nycflights13

This data consists of information about all flights that departed NYC in 2013. Working with two datasets: flights and airlines.

1. Merge these two DataFrames into a single DataFrame: nycflights13.
2. Are there any missing values in this DataFrame? If so, drop them.


In [0]:
# create flights df
nycflights_df = pd.read_csv(flights_path)

# create airlines df
airlines_df = pd.read_csv(airlines_path)

# merge flights and airlines
nycflights13 = pd.merge(nycflights_df,airlines_df)

# is the merged dataset as long as the larger one from the 2 datasets  it was created from
nycflights13.shape[0] == max(nycflights_df.shape[0],airlines_df.shape[0]) 

336776

In [0]:
# are there missing values in merged df?

# resource for findng missing values: https://stackoverflow.com/questions/28199524/best-way-to-count-the-number-of-rows-with-missing-values-in-a-pandas-dataframe

'''
  .isnull() gives True for missing values. Can we count the numbe of True??
  we know that True evaluates to 1 and False to zero. Hence the sum of booleans on .isnull() can 
  give us what we want
    with .values() we can get just the row values by stripping axes labels (both row and column)
    with .ravel() we can get flatten the list of row-wise booleans that we get using .values()
    with .sum() we can take the sum of all these booleans and know how many are missing
'''
#for loop to go check each column


# counting all cells with missing values
nycflights13.isnull().values.ravel().sum()

# counting just the rows with missing values
nycflights13.shape[0] - nycflights13.dropna().shape[0]

327346

# `census`

  Work with two datasets: census_population and census_regions.

1. Merge these two DataFrames into a single DataFrame called census.

2. Re-shape census so that one column contains all population measures, and another the year attributes.

3. Group the data by year and summarize it.

4. Group the data by region, division and year and summarize it.




In [0]:
# create c-p df
pop_df = pd.read_csv(state_pop_path)

# create c-d df
divisions_df = pd.read_csv(census_divisions_path)

# merge flights and airlines
census = pd.merge(pop_df,divisions_df)

# is the merged dataset as long as the larger one from the 2 datasets  it was created from
census.shape[0] == max(pop_df.shape[0],divisions_df.shape[0]) 

# key variable for merge was: 
pop_df.columns.intersection(divisions_df.columns)

Index(['state'], dtype='object')

In [0]:
# melt to get 2 column-df with pop and year attributes, by state
melted = pd.melt(census, id_vars=['state'], value_vars=census.columns[1:8], var_name='year', value_name='population')
melted.sort_values(by=["state","year"]) 

,state,year,population
0,Alabama,2010,4785492
51,Alabama,2011,4799918
102,Alabama,2012,4815960
153,Alabama,2013,4829479
204,Alabama,2014,4843214
...,...,...,...
152,Wyoming,2012,576765
203,Wyoming,2013,582684
254,Wyoming,2014,583642
305,Wyoming,2015,586555


In [0]:
# group melted data by year and summarize it
melted.groupby(['year']).describe()

population                              ...                                  
          count          mean           std  ...        50%        75%         max
year                                         ...                                  
2010       51.0  6.065651e+06  6.839536e+06  ...  4348662.0  6654375.0  37332685.0
2011       51.0  6.111046e+06  6.907730e+06  ...  4369354.0  6717221.5  37676861.0
2012       51.0  6.156831e+06  6.974942e+06  ...  4384799.0  6776617.0  38011074.0
2013       51.0  6.200096e+06  7.038774e+06  ...  4400477.0  6837396.0  38335203.0
2014       51.0  6.246342e+06  7.110811e+06  ...  4413057.0  6902053.5  38680810.0
2015       51.0  6.292091e+06  7.181603e+06  ...  4424611.0  6988927.5  38993940.0
2016       51.0  6.335834e+06  7.243849e+06  ...  4436974.0  7109535.5  39250017.0

[7 rows x 8 columns]

In [0]:
# Group the data by region, division and year and summarize it
census.groupby(['region','division']).mean() # can use .describe() for additional stats

2010  ...          2016
region    division                          ...              
Midwest   East North Central  9.288169e+06  ...  9.351195e+06
          West North Central  2.933965e+06  ...  3.026494e+06
Northeast Middle Atlantic     1.363957e+07  ...  1.382466e+07
          New England         2.411557e+06  ...  2.455921e+06
South     East South Central  4.615287e+06  ...  4.735048e+06
          South Atlantic      6.659007e+06  ...  7.102590e+06
          West South Central  9.117726e+06  ...  9.864018e+06
West      Mountain            2.765811e+06  ...  2.981883e+06
          Pacific             9.998387e+06  ...  1.056039e+07

[9 rows x 7 columns]